In [2]:
#convert_masks.py
import os
import numpy as np
from PIL import Image
from tqdm import tqdm  # Progress bar

# Input folder: RGB masks
input_mask_dir = "DeepGlobe/train"

# Output folder: class-index masks
output_mask_dir = "DeepGlobe/train_converted"
os.makedirs(output_mask_dir, exist_ok=True)

# RGB to class index mapping
rgb_to_class = {
    (0, 255, 255): 0,       # Urban
    (255, 255, 0): 1,       # Agriculture
    (255, 0, 255): 2,       # Rangeland
    (0, 255, 0): 3,         # Forest
    (0, 0, 255): 4,         # Water
    (255, 255, 255): 5,     # Barren
    (0, 0, 0): 6            # Unknown
}

# Gather mask filenames
mask_files = [f for f in os.listdir(input_mask_dir) if f.endswith(("_mask.jpeg", "_mask.png"))]

# Convert masks with progress bar
for filename in tqdm(mask_files, desc="🔄 Converting masks"):
    mask_path = os.path.join(input_mask_dir, filename)
    rgb_mask = np.array(Image.open(mask_path).convert("RGB"))

    class_mask = np.zeros(rgb_mask.shape[:2], dtype=np.uint8)

    for rgb, class_idx in rgb_to_class.items():
        match = np.all(rgb_mask == rgb, axis=-1)
        class_mask[match] = class_idx

    output_file = filename.replace("_mask", "_mask_class").replace(".jpeg", ".png")
    output_path = os.path.join(output_mask_dir, output_file)
    Image.fromarray(class_mask).save(output_path)

In [ ]:
#preprocessing.py
import os
from PIL import Image
import numpy as np

# Tile size & stride
TILE_SIZE = 256
STRIDE = 256

# Input folders
IMAGE_DIR = "DeepGlobe/temp"
LABEL_DIR = "DeepGlobe/temp_converted"

# Output folders
TILE_IMG_DIR = "data/tiles/images"
TILE_LABEL_DIR = "data/tiles/labels"

def create_required_directories():
    os.makedirs(TILE_IMG_DIR, exist_ok=True)
    os.makedirs(TILE_LABEL_DIR, exist_ok=True)

def tile_image_and_mask(image_path, mask_path, tile_size=TILE_SIZE, stride=STRIDE):
    image = np.array(Image.open(image_path).convert("RGB"))
    mask = np.array(Image.open(mask_path))

    h, w = image.shape[:2]
    tile_id = 0
    img_name = os.path.basename(image_path).split('_')[0]

    for y in range(0, h - tile_size + 1, stride):
        for x in range(0, w - tile_size + 1, stride):
            img_tile = image[y:y + tile_size, x:x + tile_size]
            mask_tile = mask[y:y + tile_size, x:x + tile_size]

            img_tile_path = os.path.join(TILE_IMG_DIR, f"{img_name}_tile_{tile_id}.png")
            mask_tile_path = os.path.join(TILE_LABEL_DIR, f"{img_name}_tile_{tile_id}.png")

            Image.fromarray(img_tile).save(img_tile_path)
            Image.fromarray(mask_tile).save(mask_tile_path)
            tile_id += 1

    print(f"✅ Tiled {img_name}: {tile_id} patches")

def batch_tile_dataset():
    create_required_directories()
    all_files = sorted(os.listdir(IMAGE_DIR))
    sat_images = [f for f in all_files if f.endswith('_sat.jpg')]

    print("🧾 Found satellite images:", len(sat_images))

    for sat_file in sat_images:
        base_name = sat_file.replace('_sat.jpg', '')  
        mask_file = f"{base_name}_mask_class.png"
        sat_path = os.path.join(IMAGE_DIR, sat_file)
        mask_path = os.path.join(LABEL_DIR, mask_file)

        print(f"🔄 Processing: {sat_file}")

        if os.path.exists(mask_path):
            tile_image_and_mask(sat_path, mask_path)
        else:
            print(f"⚠️ Mask missing for {sat_file}")
if __name__ == "__main__":
    batch_tile_dataset()

In [ ]:
#mlp_classifier_task1.py
import os
from PIL import Image
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, classification_report
from tqdm import tqdm
from collections import Counter
import joblib

# Tile folders
TILE_IMG_DIR = "data/tiles/images"
TILE_LABEL_DIR = "data/tiles/labels"

# Collect file names
tile_files = sorted(os.listdir(TILE_IMG_DIR))
print(f" Found {len(tile_files)} tiles")

X, y = [], []

def extract_features(image_path):
    img = Image.open(image_path).convert("RGB")
    img = np.array(img)
    r, g, b = img[:, :, 0].mean(), img[:, :, 1].mean(), img[:, :, 2].mean()
    return [r, g, b]

def extract_label(mask_path):
    mask = np.array(Image.open(mask_path))
    flat = mask.flatten()
    flat = flat[flat > 0]  # ignore class 0 (background)
    if len(flat) == 0:
        return 0
    return int(np.bincount(flat).argmax())

# Load features and labels
for tile in tqdm(tile_files, desc=" Loading tiles"):
    img_path = os.path.join(TILE_IMG_DIR, tile)
    mask_path = os.path.join(TILE_LABEL_DIR, tile)

    if os.path.exists(mask_path):
        features = extract_features(img_path)
        label = extract_label(mask_path)

        X.append(features)
        y.append(label)

X = np.array(X)
y = np.array(y)

print(f"\n✅ Dataset shape: {X.shape}, Labels: {Counter(y)}")

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train MLP
mlp = MLPClassifier(hidden_layer_sizes=(30, 15, 20), max_iter=500, random_state=42)
mlp.fit(X_train, y_train)

# Evaluate
y_pred = mlp.predict(X_test)
print(f"\n✅ MLP Accuracy: {accuracy_score(y_test, y_pred):.4f}")
print("\n📊 Classification Report:")
print(classification_report(y_test, y_pred))

# Save model
joblib.dump(mlp, "mlp_landcover_model.pkl")
print("✅ MLP model saved as mlp_landcover_model.pkl")


In [ ]:
#ensemble_classifier_task2.py
mport os
import numpy as np
from PIL import Image
from collections import Counter
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from tqdm import tqdm

# Paths
IMAGE_DIR = "data/tiles/images"
LABEL_DIR = "data/tiles/labels"

def extract_features(image_path):
    img = Image.open(image_path).convert("RGB")
    img = np.array(img)
    r_mean = img[:, :, 0].mean()
    g_mean = img[:, :, 1].mean()
    b_mean = img[:, :, 2].mean()
    return [r_mean, g_mean, b_mean]

def extract_label(mask_path):
    mask = Image.open(mask_path)
    mask = np.array(mask)
    mask = mask[mask != 0]  # ignore background
    if len(mask) == 0:
        return 0
    return int(np.bincount(mask.flatten()).argmax())

# Data Loading
X, y = [], []
image_files = sorted(os.listdir(IMAGE_DIR))

print(" Loading tiles:", end=" ")
for file in tqdm(image_files):
    img_path = os.path.join(IMAGE_DIR, file)
    mask_path = os.path.join(LABEL_DIR, file)

    if os.path.exists(mask_path):
        features = extract_features(img_path)
        label = extract_label(mask_path)
        X.append(features)
        y.append(label)

print(f"\n✅ Dataset shape: {np.array(X).shape}, Labels: {Counter(y)}")

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# ✅ Random Forest Classifier
rf = RandomForestClassifier(n_estimators=100, max_depth=15, random_state=42)
rf.fit(X_train, y_train)

# Predictions
y_pred = rf.predict(X_test)

# Evaluation
accuracy = accuracy_score(y_test, y_pred)
print(f"\n✅ Random Forest Accuracy: {accuracy:.4f}\n")
print("📊 Classification Report:")
print(classification_report(y_test, y_pred))

In [ ]:
#cnn_classifier_task3.py
import os
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.utils import to_categorical
from collections import Counter
from tqdm import tqdm
import matplotlib.pyplot as plt

# Tile folders
IMAGE_DIR = "data/tiles/images"
LABEL_DIR = "data/tiles/labels"

# Parameters
IMG_SIZE = 64  # resized down from 256x256
NUM_CLASSES = 7

def load_data():
    X, y = [], []

    all_images = sorted(os.listdir(IMAGE_DIR))

    for img_file in tqdm(all_images, desc=" Loading tiles"):
        label_file = img_file.replace(".png", ".png")
        img_path = os.path.join(IMAGE_DIR, img_file)
        label_path = os.path.join(LABEL_DIR, label_file)

        if not os.path.exists(label_path):
            continue

        # Load and resize image & mask
        img = np.array(Image.open(img_path).resize((IMG_SIZE, IMG_SIZE))) / 255.0
        mask = np.array(Image.open(label_path).resize((IMG_SIZE, IMG_SIZE), resample=Image.NEAREST))

        # Use the most common pixel value in the mask as the label
        label = int(Counter(mask.flatten()).most_common(1)[0][0])

        X.append(img)
        y.append(label)

    return np.array(X), np.array(y)

# Load dataset
X, y = load_data()
print(f"\n✅ Dataset shape: {X.shape}, Labels: {Counter(y)}")

# One-hot encode labels
y_cat = to_categorical(y, NUM_CLASSES)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y_cat, test_size=0.2, random_state=42)

# Build CNN model
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 3)),
    MaxPooling2D((2,2)),

    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D((2,2)),

    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D((2,2)),

    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(NUM_CLASSES, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

# Train the model
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=64)

# Evaluate
loss, acc = model.evaluate(X_test, y_test)
print(f"\n✅ CNN Accuracy: {acc:.4f}")

# Predict class labels on test data
y_pred_probs = model.predict(X_test)
y_pred = np.argmax(y_pred_probs, axis=1)
y_true = np.argmax(y_test, axis=1)

# Class ID to label mapping
class_labels = {
    0: "Urban",
    1: "Agriculture",
    2: "Rangeland",
    3: "Forest",
    4: "Water",
    5: "Barren",
    6: "Unknown"
}

# Visualize 5 random predictions
plt.figure(figsize=(15, 6))
for i in range(5):
    idx = np.random.randint(0, len(X_test))
    img = X_test[idx]

    plt.subplot(1, 5, i+1)
    plt.imshow(img)
    plt.title(f"Pred: {class_labels[y_pred[idx]]}\nTrue: {class_labels[y_true[idx]]}")
    plt.axis("off")

plt.tight_layout()
plt.show()

# Save the trained model
model.save("cnn_landcover_model.h5")
print("💾 Model saved as cnn_landcover_model.h5")

In [ ]:
#clustering_task4.py
import os
import numpy as np
from PIL import Image
from tqdm import tqdm
from sklearn.cluster import KMeans, DBSCAN
from sklearn.metrics import adjusted_rand_score, silhouette_score
from collections import Counter

# Paths
image_dir = "data/tiles/images"
label_dir = "data/tiles/labels"

def extract_features(image_path):
    img = np.array(Image.open(image_path).convert("RGB"))
    return img.mean(axis=(0, 1))  # Mean R, G, B

def extract_label(mask_path):
    mask = np.array(Image.open(mask_path))
    flat = mask.flatten()
    flat = flat[flat > 0]
    return int(np.bincount(flat).argmax()) if len(flat) > 0 else 0

# Load data
X, y_true = [], []
image_files = sorted(os.listdir(image_dir))

print(f"📦 Found {len(image_files)} image tiles")

for file in tqdm(image_files, desc="🔄 Loading tiles"):
    img_path = os.path.join(image_dir, file)
    mask_path = os.path.join(label_dir, file)
    if os.path.exists(mask_path):
        X.append(extract_features(img_path))
        y_true.append(extract_label(mask_path))

X = np.array(X)
y_true = np.array(y_true)

print("✅ Data loaded:", X.shape)

# KMeans clustering
print("\n🔍 Running KMeans...")
kmeans = KMeans(n_clusters=7, random_state=42, n_init='auto')
y_kmeans = kmeans.fit_predict(X)

ari_kmeans = adjusted_rand_score(y_true, y_kmeans)
sil_kmeans = silhouette_score(X, y_kmeans)

# DBSCAN clustering
print("\n🔍 Running DBSCAN...")
dbscan = DBSCAN(eps=20, min_samples=5)
y_dbscan = dbscan.fit_predict(X)

ari_dbscan = adjusted_rand_score(y_true, y_dbscan)
sil_dbscan = silhouette_score(X, y_dbscan)

# Report results
print("\n📊 Clustering Evaluation Report:")
print(f"KMeans - ARI: {ari_kmeans:.4f}, Silhouette Score: {sil_kmeans:.4f}")
print(f"DBSCAN - ARI: {ari_dbscan:.4f}, Silhouette Score: {sil_dbscan:.4f}")

In [ ]:
#import Matplotlib.pyplot as plt
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image

# Load a mask file
mask = np.array(Image.open("DeepGlobe/temp_converted/119_mask_class.png"))

# Define color map for 7 classes
colors = [
    (0, 255, 255),     # Urban
    (255, 255, 0),     # Agriculture
    (255, 0, 255),     # Rangeland
    (0, 255, 0),       # Forest
    (0, 0, 255),       # Water
    (255, 255, 255),   # Barren
    (0, 0, 0)          # Unknown
]

# Convert class mask to RGB image
rgb_mask = np.zeros((mask.shape[0], mask.shape[1], 3), dtype=np.uint8)
for class_idx, color in enumerate(colors):
    rgb_mask[mask == class_idx] = color

plt.imshow(rgb_mask)
plt.title("Visualized Class Mask")
plt.axis('off')
plt.show()

In [ ]:
#visualize_preictions.py
import os
import numpy as np
from PIL import Image
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from collections import Counter
from tensorflow.keras.models import load_model
from tensorflow.keras.utils import to_categorical
from tqdm import tqdm
import joblib
from sklearn.neural_network import MLPClassifier

# === Common Settings ===
IMAGE_DIR = "data/tiles/images"
LABEL_DIR = "data/tiles/labels"
IMG_SIZE = 64
NUM_CLASSES = 7
CLASS_NAMES = ["Urban", "Agriculture", "Rangeland", "Forest", "Water", "Barren", "Unknown"]

# === CNN Confusion Matrix ===
print("\n📦 Running CNN confusion matrix evaluation...")

cnn_model = load_model("cnn_landcover_model.h5")

X_cnn, y_true_cnn = [], []
files = sorted(os.listdir(IMAGE_DIR))

for file in tqdm(files, desc="🔄 Loading tiles for CNN"):
    img_path = os.path.join(IMAGE_DIR, file)
    label_path = os.path.join(LABEL_DIR, file)

    if not os.path.exists(label_path):
        continue

    img = np.array(Image.open(img_path).resize((IMG_SIZE, IMG_SIZE))) / 255.0
    mask = np.array(Image.open(label_path).resize((IMG_SIZE, IMG_SIZE), resample=Image.NEAREST))
    label = int(Counter(mask.flatten()).most_common(1)[0][0])

    X_cnn.append(img)
    y_true_cnn.append(label)

X_cnn = np.array(X_cnn)
y_true_cnn = np.array(y_true_cnn)

y_pred_cnn = np.argmax(cnn_model.predict(X_cnn, batch_size=64), axis=1)

cm_cnn = confusion_matrix(y_true_cnn, y_pred_cnn, labels=range(NUM_CLASSES))
disp_cnn = ConfusionMatrixDisplay(confusion_matrix=cm_cnn, display_labels=CLASS_NAMES)
plt.figure(figsize=(10, 8))
disp_cnn.plot(cmap="Blues", xticks_rotation=45)
plt.title("📊 Confusion Matrix: CNN Land Cover Prediction")
plt.tight_layout()
plt.show()

# === MLP Confusion Matrix ===
print("\n📦 Running MLP confusion matrix evaluation...")

mlp_model = joblib.load("mlp_landcover_model.pkl")

def extract_mlp_features(image_path):
    img = Image.open(image_path).convert("RGB")
    img = np.array(img)
    return [img[:, :, 0].mean(), img[:, :, 1].mean(), img[:, :, 2].mean()]

def extract_mlp_label(mask_path):
    mask = np.array(Image.open(mask_path))
    flat = mask.flatten()
    flat = flat[flat > 0]
    return int(np.bincount(flat).argmax()) if len(flat) > 0 else 0

X_mlp, y_true_mlp = [], []
for file in tqdm(files, desc="🔄 Loading tiles for MLP"):
    img_path = os.path.join(IMAGE_DIR, file)
    mask_path = os.path.join(LABEL_DIR, file)

    if os.path.exists(mask_path):
        X_mlp.append(extract_mlp_features(img_path))
        y_true_mlp.append(extract_mlp_label(mask_path))

X_mlp = np.array(X_mlp)
y_true_mlp = np.array(y_true_mlp)
y_pred_mlp = mlp_model.predict(X_mlp)

cm_mlp = confusion_matrix(y_true_mlp, y_pred_mlp, labels=range(NUM_CLASSES))
disp_mlp = ConfusionMatrixDisplay(confusion_matrix=cm_mlp, display_labels=CLASS_NAMES)
plt.figure(figsize=(10, 8))
disp_mlp.plot(cmap="Purples", xticks_rotation=45)
plt.title("📊 Confusion Matrix: MLP Land Cover Prediction")
plt.tight_layout()
plt.show()